In [1]:
import pandas as pd

df_genome_scores=pd.read_csv('../../dataset/ml-20m/genome-scores.csv')
df_genome_tags=pd.read_csv('../../dataset/ml-20m/genome-tags.csv')
df_links=pd.read_csv('../../dataset/ml-20m/links.csv')
df_movies=pd.read_csv('../../dataset/ml-20m/movies.csv')
df_ratings=pd.read_csv('../../dataset/ml-20m/ratings.csv')
df_tags=pd.read_csv('../../dataset/ml-20m/tags.csv')

In [2]:
df_genome_scores.info()
df_genome_scores.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
movieId      int64
tagId        int64
relevance    float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [3]:
df_genome_tags.describe()

,tagId
count,1128.000000
mean,564.500000
std,325.769857
min,1.000000
25%,282.750000
50%,564.500000
75%,846.250000
max,1128.000000


In [4]:
#df_links
print(len(df_links.movieId.unique()))
#27278 films dans cette base de donnée

27278


In [5]:
#df_movies
df_movies.info()
print(len(df_movies.movieId.unique()))
#27278 films dans cette base de donnée

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId    27278 non-null int64
title      27278 non-null object
genres     27278 non-null object
dtypes: int64(1), object(2)
memory usage: 639.4+ KB
27278


In [6]:
#df_ratings
df_ratings.info()
print(len(df_ratings.movieId.unique()))
#26744 films dans cette base de donnée

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 610.4 MB
26744


In [7]:
#df_tags
df_tags.info()
print(len(df_tags.movieId.unique()))
#19545 fims dans cette base de donnée

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
userId       465564 non-null int64
movieId      465564 non-null int64
tag          465548 non-null object
timestamp    465564 non-null int64
dtypes: int64(3), object(1)
memory usage: 14.2+ MB
19545


In [8]:
# A part pour les commentaires (moins utile opur nos algo), 
# le plus petit nombre de films est 26744 dans le fichier MovieLens-ratings
# Donc on trie toutes les autres bases de données sur cette base de films
# On numérote nos films de 1 à 26744 

df_ref_movie=df_ratings.groupby("movieId").count()
df_ref_movie["movieId_2"]=range(1,26745)
df_ref_movie.head()

#Enregistrement d'un df qui conserve toutes les correspondances Id
df_ref_movie=df_ref_movie.drop(["userId","rating","timestamp"], axis=1)
df_ref_movie.head()
df_ref_movie.to_csv('../new_dataset/correspondances_transitoire_Id_movie', sep=',', columns=df_ref_movie.columns)
#Avant de continuer plus loin, j'ai mis à jour le fichier correspondances_Id_movie avec la base de donnée IMDB, car celle-ci ne contient pas tous les films de MovieLens
#Donc les movieId_ref sont les ids définitifs. On a donc 23608 films.

In [9]:
#Ajout de movieId_ref à df_ratings
df_ref_movie=pd.read_csv('../new_dataset/correspondances_Id_movie')
df_ref_movie.index=df_ref_movie.movieId
df_ratings.index=df_ratings.movieId
df_ratings=df_ratings.join(df_ref_movie.movieId_ref)

In [10]:
#Tri de df_ratings
df_ratings.head()
df_ratings=df_ratings.dropna(subset=["movieId_ref"])
print(len(df_ratings.movieId.unique()))
df_ratings.head()

23608


,userId,movieId,rating,timestamp,movieId_ref
movieId,,,,,
1,3,1,4.0,944919407,1.0
1,6,1,5.0,858275452,1.0
1,8,1,4.0,833981871,1.0
1,10,1,4.0,943497887,1.0
1,11,1,4.5,1230858821,1.0


In [11]:
#Création stats par user

user_stat=df_ratings.groupby("userId").count()
rating_mean=df_ratings.groupby("userId").mean()
rating_ec=df_ratings.groupby("userId").std()
rating_mean=rating_mean.drop(['movieId', 'timestamp'], axis=1)
rating_ec=rating_ec.drop(['movieId', 'timestamp'], axis=1)
user_stat=user_stat.join(rating_mean, on=user_stat.index, rsuffix=2)
user_stat=user_stat.join(rating_ec, on=user_stat.index, rsuffix=3)
user_stat=user_stat.drop(["rating", "timestamp", "movieId_ref", "movieId_ref2", "movieId_ref3"], axis=1)
user_stat=user_stat.rename(columns={'movieId' : 'nb_votes', 'rating2' : 'moy_votes', 'rating3':'std_votes'})

In [12]:
user_stat.head()

,nb_votes,moy_votes,std_votes
userId,,,
1,175,3.742857,0.382284
2,60,4.000000,1.073565
3,185,4.129730,0.911540
4,28,3.571429,0.790151
5,64,4.296875,0.937401


In [13]:
#Création stats par movie

movie_stat=df_ratings.groupby("movieId_ref").count()
rating_mean=df_ratings.groupby("movieId").mean()
rating_ec=df_ratings.groupby("movieId").std()
rating_mean=rating_mean.drop(['userId', 'timestamp'], axis=1)
rating_ec=rating_ec.drop(['userId', 'timestamp'], axis=1)
movie_stat=movie_stat.join(rating_mean, on=movie_stat.index, rsuffix=2)
movie_stat=movie_stat.join(rating_ec, on=movie_stat.index, rsuffix=3)
movie_stat=movie_stat.drop(['rating', 'timestamp', 'movieId', 'movieId_ref', 'movieId_ref3'], axis=1)


#Rajout des notes de imbd

movie_df=pd.read_csv('../new_dataset/movie_df.csv', index_col=0)
movie_df=movie_df[["numVotes","averageRating"]]
movie_df.head()
movie_stat=movie_stat.join(movie_df, on=movie_stat.index)

movie_stat=movie_stat.rename(columns={'userId' : 'ML_nb_votes', 'rating2' : 'ML_moy_votes', 'rating3':'ML_std_votes', 'averageRating':'IMDB_moy_votes', 'numVotes':'IMDB_nb_votes'})


ValueError: 'movieId' is both an index level and a column label, which is ambiguous.

In [ ]:
movie_stat.head()

In [ ]:
#enregistrement des 2 df user_stat et movie_stat
user_stat.to_csv('../new_dataset/user_stat_ratings.csv', sep=',', columns=user_stat.columns)
movie_stat.to_csv('../new_dataset/movie_stat_ratings.csv', sep=',', columns=movie_stat.columns)
df_ratings.to_csv('../new_dataset/ratings.csv', sep=',', columns=df_ratings.columns)